In [1]:
import math
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from importlib import reload
%matplotlib qt

In [2]:
df = pd.read_csv("MMM.csv")

fig = plt.figure(figsize=(12, 10), dpi=100)
ax = fig.add_subplot(111)

for x in df["body_slip"].unique():
    df2 = df[df["body_slip"] == x]
    lat_accel = list(df2["y_double_dot"]/9.81)
    yaw_accel = list(df2["yaw_acceleration"])
    mid_point = int(len(lat_accel)/2)

    ax.plot(lat_accel, yaw_accel, 'r')
    #ax.text(lat_accel[mid_point], yaw_accel[mid_point], '{0:.2f}'.format(x*180/math.pi))

for x in df["steered_angle"].unique():
    if True:#x == 0:
        df2 = df[(df["steered_angle"] == x)]
        ax.plot(df2["y_double_dot"]/9.81, df2["yaw_acceleration"], 'b')
ax.grid()
ax.set_ylabel("Yaw Acceleration (rad/s^2)")
ax.set_xlabel("Lateral Acceleration (g)")

abs(df["yaw_acceleration"]).min()

2.6364401059560078e-30

In [3]:
import math
plt.scatter(df["y_double_dot"], df["front_left_tire_centric_forces_2"], label = "Front Right")
plt.scatter(df["y_double_dot"], df["front_right_tire_centric_forces_2"], label = "Front Left")
plt.scatter(df["y_double_dot"], df["rear_left_tire_centric_forces_2"], label = "Rear Right")
plt.scatter(df["y_double_dot"], df["rear_right_tire_centric_forces_2"], label = "Rear Left")
plt.title("Normal Force vs Lateral Accel")
plt.grid()
plt.ylabel("Normal Force (N)")
plt.xlabel("Lateral Acceleration (m/s^2)")
plt.legend()

In [4]:
plt.scatter(df["y_double_dot"], df["front_left_vehicle_centric_forces_1"], label = "Front Right")
plt.scatter(df["y_double_dot"], df["front_right_vehicle_centric_forces_1"], label = "Front Left")
plt.scatter(df["y_double_dot"], df["rear_left_vehicle_centric_forces_1"], label = "Rear Right")
plt.scatter(df["y_double_dot"], df["rear_right_vehicle_centric_forces_1"], label = "Rear Left")
plt.title("Lateral Force vs Lateral Accel")
plt.grid()
plt.ylabel("Lateral Force (N)")
plt.xlabel("Lateral Acceleration (m/s^2)")
plt.legend()

In [5]:
plt.scatter(df["y_double_dot"], df["front_left_z_c"], label = "Front Right")
plt.scatter(df["y_double_dot"], df["front_right_z_c"], label = "Front Left")
plt.scatter(df["y_double_dot"], df["rear_left_z_c"], label = "Rear Right")
plt.scatter(df["y_double_dot"], df["rear_right_z_c"], label = "Rear Left")
plt.title("Vertical Chassis Displacement vs Lateral Accel")
plt.grid()
plt.ylabel("Vertical Chassis Displacement (m)")
plt.xlabel("Lateral Acceleration (m/s^2)")
plt.legend()

In [6]:
plt.scatter(df["y_double_dot"], df["front_left_f_roll"], label = "Front Right")
plt.scatter(df["y_double_dot"], df["front_right_f_roll"], label = "Front Left")
plt.scatter(df["y_double_dot"], df["rear_left_f_roll"], label = "Rear Right")
plt.scatter(df["y_double_dot"], df["rear_right_f_roll"], label = "Rear Left")
plt.title("Vertical Force vs Lateral Accel")
plt.grid()
plt.ylabel("Vertical Force Roll (N)")
plt.xlabel("Lateral Acceleration (m/s^2)")
plt.legend()

In [7]:
plt.scatter(df["y_double_dot"], df["front_left_f_heave"], label = "Front Right")
plt.scatter(df["y_double_dot"], df["front_right_f_heave"], label = "Front Left")
plt.scatter(df["y_double_dot"], df["rear_left_f_heave"], label = "Rear Right")
plt.scatter(df["y_double_dot"], df["rear_right_f_heave"], label = "Rear Left")
plt.title("Vertical Force Ride vs Lateral Accel")
plt.grid()
plt.ylabel("Vertical Force Ride (N)")
plt.xlabel("Lateral Acceleration (m/s^2)")
plt.legend()

In [8]:
plt.scatter(df["y_double_dot"], df["steered_angle"])
plt.title("Vertical Force Ride vs Lateral Accel")
plt.grid()
plt.ylabel("Vertical Force Ride (N)")
plt.xlabel("Lateral Acceleration (m/s^2)")
plt.legend()

In [9]:
fig = plt.figure(figsize=(12, 10), dpi=100)
ax = fig.add_subplot(111,projection='3d')

# for velocity in df["x_dot"].unique():
velocity = 0
df1 = df[df["x_dot"] == velocity]
for x in df["body_slip"].unique():
    df2 = df[df["body_slip"] == x]
    lat_accel = list(df2["y_double_dot"]/9.81)
    yaw_accel = list(df2["yaw_acceleration"])
    mid_point = int(len(lat_accel)/2)

    ax.plot(lat_accel, yaw_accel, velocity, 'r')
    #ax.text(lat_accel[mid_point], yaw_accel[mid_point], '{0:.2f}'.format(x*180/math.pi))

for x in df["steered_angle"].unique():
    if True:#x == 0:
        df2 = df[(df["steered_angle"] == x)]
        ax.plot(df2["y_double_dot"]/9.81, df2["yaw_acceleration"], velocity, 'b')
ax.grid()
ax.set_ylabel("Yaw Acceleration (rad/s^2)")
ax.set_xlabel("Lateral Acceleration (g)")
ax.set_zlabel("Longitudinal Velocity (m/s)")

Text(0.5, 0, 'Longitudinal Velocity (m/s)')

In [11]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d


fig = plt.figure(figsize=(12, 10), dpi=100)
ax = fig.add_subplot(111)

colors = ["red", "blue", "green", "orange"]
colors = np.tile(colors,10)
print(colors)
i = 0
for velocity in df["x_dot"].unique():
    # rgb_random = np.random.rand(3,)
    rgb_random = colors[i]
    # Plot the raw MMM points
    df1 = df[df["x_dot"] == velocity]
    # ax.scatter(df1["y_double_dot"]/9.81, df1["yaw_acceleration"], velocity, c=[rgb_random])

    # Convert df into 2d python array
    npoints = df1["y_double_dot"].to_numpy().shape[0]
    arr = np.empty(shape=(npoints,2))
    arr[:,0] = df1["y_double_dot"].to_numpy()/9.81
    arr[:,1] = df1["yaw_acceleration"].to_numpy()

    # Create the hull
    hull = ConvexHull(arr)
    # hull.simplices[0][1]

    for simplex in hull.simplices:
        ax.plot(arr[simplex,0], arr[simplex,1], 'k-', c=rgb_random)
    i+=1

ax.grid()
ax.set_ylabel("Yaw Acceleration (rad/s^2)")
ax.set_xlabel("Lateral Acceleration (g)")


['red' 'blue' 'green' 'orange' 'red' 'blue' 'green' 'orange' 'red' 'blue'
 'green' 'orange' 'red' 'blue' 'green' 'orange' 'red' 'blue' 'green'
 'orange' 'red' 'blue' 'green' 'orange' 'red' 'blue' 'green' 'orange'
 'red' 'blue' 'green' 'orange' 'red' 'blue' 'green' 'orange' 'red' 'blue'
 'green' 'orange']


C:\Users\Daniel\AppData\Local\Temp/ipykernel_29924/1253162496.py:29: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k-" (-> color='k'). The keyword argument will take precedence.
  ax.plot(arr[simplex,0], arr[simplex,1], 'k-', c=rgb_random)
C:\Users\Daniel\AppData\Local\Temp/ipykernel_29924/1253162496.py:29: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k-" (-> color='k'). The keyword argument will take precedence.
  ax.plot(arr[simplex,0], arr[simplex,1], 'k-', c=rgb_random)
C:\Users\Daniel\AppData\Local\Temp/ipykernel_29924/1253162496.py:29: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k-" (-> color='k'). The keyword argument will take precedence.
  ax.plot(arr[simplex,0], arr[simplex,1], 'k-', c=rgb_random)
C:\Users\Daniel\AppData\Local\Temp/ipykernel_29924/1253162496.py:29: UserWarning: color is redundantly defined by the 'color' keyword ar

Text(0.5, 0, 'Lateral Acceleration (g)')